In [25]:
import os
import random
import shutil
import pandas as pd
from PIL import Image

### Kitti dataset to yolo dataset
- 매치되는 이미지 및 라벨을 읽어옴
- 이미지의 경우 png to jpg 진행
- 라벨데이터의 경우 class index, bounding box convert 수행
- 기존 이미지 데이터의 이름 + 10자리 알파벳, 숫자 난수 조합을 붙여 각각 이미지 및 라벨데이터에 같은 이름으로 변환해서 저장

In [35]:
# class index 지정
class_mapping = {
    'Car': 1,
    'Van': 2,
    'Truck': 3,
    'Pedestrian': 4,
    'Person_sitting': 4,
    'Person' : 4,
    'Cyclist': 5,
    'Tram': 6,
}

In [36]:
image_folder = '/ultralytics/examples/Work_dir/kitti/image_02'
label_folder = '/ultralytics/examples/Work_dir/kitti/label_02'
output_image_folder = '/ultralytics/examples/Work_dir/kitti_dataset/images'
output_label_folder = '/ultralytics/examples/Work_dir/kitti_dataset/labels'

In [37]:
image_subfolders = sorted(os.listdir(image_folder))

In [38]:
for subfolder in image_subfolders:
    subfolder_path = os.path.join(image_folder, subfolder)
    if os.path.isdir(subfolder_path):
        image_files = sorted(os.listdir(subfolder_path))
        
        lbl_file = f"{subfolder}.txt"
        lbl_path = os.path.join(label_folder, lbl_file)
        
        label_df = pd.DataFrame(columns = ['img_num', 'others_1', 'class', 'others_2', 'others_3', 
                                          'others_4',  'box_1', 'box_2', 'box_3', 'box_4', 'others_5', 
                                          'others_6', 'others_7', 'others_8', 'others_9', 'others_10', 'others_11'])
        
        lbl_f = open(lbl_path, 'r')
        for lab_num, line in enumerate(lbl_f):
            parts = line.split()
            label_df.loc[lab_num] = parts
            
        label_df['img_num'] =label_df['img_num'].astype('int64')
        label_df['box_1'] =label_df['box_1'].astype('float64')
        label_df['box_2'] =label_df['box_2'].astype('float64')
        label_df['box_3'] =label_df['box_3'].astype('float64')
        label_df['box_4'] =label_df['box_4'].astype('float64')
        
        img_num = 0
        for img_file in image_files:
            img_path = os.path.join(subfolder_path, img_file)
            
            print(img_file)
            
            if os.path.isfile(img_path):
                print(img_file)
                random_str = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=10))
                
                new_img_name = f"{os.path.splitext(img_file)[0]}_{random_str}.jpg"
                new_img_path = os.path.join(output_image_folder, new_img_name)
                
                new_lbl_name = f"{os.path.splitext(img_file)[0]}_{random_str}.txt"
                new_lbl_path = os.path.join(output_label_folder, new_lbl_name)

                im = 0
                im = Image.open(img_path)
                img_width, img_height = im.size
                
                im.save(new_img_path, 'jpeg')
                os.remove(img_path)
                                
                now_check_index = list(label_df.index[(label_df['img_num'] == img_num)])
                df_now_check = label_df.loc[now_check_index]
                df_now_check = df_now_check[df_now_check['class'] != 'DontCare']
                df_now_check = df_now_check[df_now_check['class'] != 'Misc']
                df_now_check = df_now_check[df_now_check['class'] != 'None']
                
                img_num += 1
                with open(new_lbl_path, 'w') as new_lbl_f:
                    for class_name, box_1, box_2, box_3, box_4 in zip(df_now_check['class'], df_now_check['box_1'], df_now_check['box_2'], df_now_check['box_3'], df_now_check['box_4']):
                        class_index = class_mapping.get(class_name)
                        
                        x_center = (box_1 + box_3) / 2.0 
                        y_center = (box_2 + box_4) / 2.0 
                        width = box_3 - box_1
                        height = box_4 - box_2
                        
                        x_center /= img_width
                        y_center /= img_height
                        width /= img_width
                        height /= img_height
                        
                        new_lbl_f.write(f"{class_index} {x_center} {y_center} {width} {height}\n")

000000.png
000000.png
000001.png
000001.png
000002.png
000002.png
000003.png
000003.png
000004.png
000004.png
000005.png
000005.png
000006.png
000006.png
000007.png
000007.png
000008.png
000008.png
000009.png
000009.png
000010.png
000010.png
000011.png
000011.png
000012.png
000012.png
000013.png
000013.png
000014.png
000014.png
000015.png
000015.png
000016.png
000016.png
000017.png
000017.png
000018.png
000018.png
000019.png
000019.png
000020.png
000020.png
000021.png
000021.png
000022.png
000022.png
000023.png
000023.png
000024.png
000024.png
000025.png
000025.png
000026.png
000026.png
000027.png
000027.png
000028.png
000028.png
000029.png
000029.png
000030.png
000030.png
000031.png
000031.png
000032.png
000032.png
000033.png
000033.png
000034.png
000034.png
000035.png
000035.png
000036.png
000036.png
000037.png
000037.png
000038.png
000038.png
000039.png
000039.png
000040.png
000040.png
000041.png
000041.png
000042.png
000042.png
000043.png
000043.png
000044.png
000044.png
000045.png

### 랜덤 데이터 갯수 지정하여 valid 데이터로 이동

In [39]:
base_dir = '/ultralytics/examples/Work_dir/kitti_dataset'
image_dir = os.path.join(base_dir, 'images')
label_dir = os.path.join(base_dir, 'labels')

In [40]:
val_base_dir = '/ultralytics/examples/Work_dir/kitti_val_dataset'
val_image_dir = os.path.join(val_base_dir, 'images')
val_label_dir = os.path.join(val_base_dir, 'labels')

In [41]:
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
selected_files = random.sample(image_files, 1000)

In [42]:
for file_name in selected_files:
    img_path = os.path.join(image_dir, file_name)
    lbl_path = os.path.join(label_dir, file_name.replace('.jpg', '.txt'))
    
    new_img_path = os.path.join(val_image_dir, file_name)
    new_lbl_path = os.path.join(val_label_dir, file_name.replace('.jpg', '.txt'))
    
    shutil.move(img_path, new_img_path)
    shutil.move(lbl_path, new_lbl_path)